# Clasificación de imágenes entre perros y gatos
### Nota:
Instalar conda install pillow y reiniciar ambiente

In [2]:
import os, shutil


In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

### Crear carpetas de entrenamiento, prueba y validación

In [4]:
#Para volver a correr el código no es necesario crear y asignar las carpetas y las imágenes de nuevo.
#Directorio local
base_dir = '/home/aulae1//Curso_ML/cats_and_dogs_short/cats_and_dogs_short'
#Directorio en drive
#base_dir = '/content/drive/My Drive/cats_and_dogs_small_ndsl'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
#directorios de gatos
train_cats_dir = os.path.join(train_dir, 'cats')
validation_cats_dir = os.path.join(validation_dir, 'cats')
test_cats_dir = os.path.join(test_dir, 'cats')
#directorios de perros
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
test_dogs_dir = os.path.join(test_dir, 'dogs')

In [5]:
print(validation_dogs_dir)
print(train_dir)



/home/aulae1//Curso_ML/cats_and_dogs_short/cats_and_dogs_short/validation/dogs
/home/aulae1//Curso_ML/cats_and_dogs_short/cats_and_dogs_short/train


# Construir la red convolucional

In [6]:
import tensorflow as tf
from tensorflow import keras

In [7]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [8]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3))) # **
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())#*
model.add(layers.Dense(512, activation='relu'))#cantidad de datos propuestos para la capa
model.add(layers.Dense(1, activation='sigmoid'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [10]:
from tensorflow.keras import optimizers

In [11]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
             loss='binary_crossentropy',
             metrics=['accuracy'])

# Preprocesamiento de los datos

1. Leer imágenes
2. Decodificar JPG a matriz RGB de pixeles
3. Convertir puntos flotantes en tensores
4. Escalar pixeles valores[0,255]

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

ImageDataGenerator convierne automaticamente imágenes en tensores preprocesados

#### Generador en python
Un generador es un objeto que actua como un iterador, usan el operador "yield"

In [13]:
train_datagen = ImageDataGenerator(rescale=1./255) #todos los pixeles estarán entre 0 y 255
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150), #Rescala las imagenes a 150x150
    batch_size=20,
    class_mode='binary') #Solo dos clases (perro,gato)

Found 2000 images belonging to 2 classes.


In [15]:
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

Found 1000 images belonging to 2 classes.


In [16]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


fit_generator es equivalente a fit, como argumento espera un objeto del tipo generador que arroja bloques de las imágenes de entrada.
Es necesario especificar el número de pasos por época, recordemos que tenemos 2000 datos de entrenamiento y el objeto Generador toma bloque de 20, entonces 20*100 = 2000.

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=100,# Epocas
    validation_data=validation_generator,
    validation_steps=50)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 50 steps
Epoch 1/100
100/100 [==============================] - 27s 273ms/step - loss: 0.6824 - accuracy: 0.5615 - val_loss: 0.6661 - val_accuracy: 0.6210
Epoch 2/100
100/100 [==============================] - 26s 263ms/step - loss: 0.6463 - accuracy: 0.6305 - val_loss: 0.6313 - val_accuracy: 0.6300
Epoch 3/100
100/100 [==============================] - 26s 263ms/step - loss: 0.5915 - accuracy: 0.6985 - val_loss: 0.6517 - val_accuracy: 0.6310
Epoch 4/100
100/100 [==============================] - 27s 267ms/step - loss: 0.5583 - accuracy: 0.7150 - val_loss: 0.5882 - val_accuracy: 0.6830
Epoch 5/100
100/100 [==============================] - 27s 269ms/step - loss: 0.5248 - accuracy: 0.7390 - val_loss: 0.5932 - val_accuracy: 0.6800
Epoch 6/100
100/100 [==============================] - 26s 263ms/step - loss: 0.4997 - accuracy: 0.75

100/100 [==============================] - 26s 263ms/step - loss: 0.0143 - accuracy: 0.9960 - val_loss: 1.7339 - val_accuracy: 0.7410
Epoch 54/100
100/100 [==============================] - 26s 262ms/step - loss: 0.0044 - accuracy: 0.9985 - val_loss: 1.9408 - val_accuracy: 0.7230
Epoch 55/100
100/100 [==============================] - 26s 262ms/step - loss: 0.0071 - accuracy: 0.9975 - val_loss: 1.9909 - val_accuracy: 0.7310
Epoch 56/100
100/100 [==============================] - 26s 263ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 2.0409 - val_accuracy: 0.7250
Epoch 57/100
100/100 [==============================] - 26s 264ms/step - loss: 0.0046 - accuracy: 0.9990 - val_loss: 1.9053 - val_accuracy: 0.7480
Epoch 58/100
100/100 [==============================] - 26s 263ms/step - loss: 0.0193 - accuracy: 0.9950 - val_loss: 1.8447 - val_accuracy: 0.7490
Epoch 59/100
100/100 [==============================] - 27s 268ms/step - loss: 0.0023 - accuracy: 0.9995 - val_loss: 1.9737 - val_a

In [ ]:
model.save('cats_and_dogs_small_01.h5') #Salvar el modelo después del entrenamiento

# Imprimir curvas de precisión y pérdida durante el entrenamiento

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(1, len(acc) + 1)

In [ ]:
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Data augmentation

Las gráficas anteriores muestran que el modelo sufre de sobreajuste, debido a la baja cantidad de datos el modelo no es capaz de generalizar. Data augmenting es generar de forma artificial mas datos de entrenamiento de los datos existentes por medio de transformaciones aleatorias.

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

Propiedades de este generador pueden ser consultadas en: https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

- *rotation_range* es un valor en grados (0-180)
- *width_shift* y *height_shift* son rangos de los cuáles una imagen se puede trasladar vertical u horizontalmente
- *shear_range* aplicar de forma aleatoria cortes
- *zoom_range* aleatoriamente aplicar zoom
- *horizontal_flip*  voltear la imagen horizontalmente(importante cuando no se asume simetría horizonal)
- *fill_mode* llenar los pixeles nuevos que fueron creados en las transformaciones anteriores


In [ ]:
from tensorflow.keras.preprocessing import image

In [ ]:
fnames = [os.path.join(train_cats_dir, fname) for
fname in os.listdir(train_cats_dir)]

In [ ]:
img_path = fnames[99] # Se elige una imagen a aumentar

In [ ]:
img = image.load_img(img_path, target_size=(150, 150)) # lee la imagen y se ajusta al tamaño 150x150

In [ ]:
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)

In [ ]:
i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i % 4 == 0:
        break

# Crear una nueva convnet con dropout

In [ ]:
model = models.Sequential()

In [ ]:
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(lr=1e-4),
    metrics=['accuracy'])

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=1,
    validation_data=validation_generator,
    validation_steps=50)

In [ ]:
model.save('cats_and_dogs_small_02.h5')

In [ ]:
acc1 = history.history['accuracy']
val_acc1 = history.history['val_accuracy']
loss1 = history.history['loss']
val_loss1 = history.history['val_loss']

In [ ]:
epochs = range(1, len(acc1) + 1)

In [ ]:
plt.plot(epochs, acc1, 'bo', label='Training acc')
plt.plot(epochs, val_acc1, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss1, 'bo', label='Training loss')
plt.plot(epochs, val_loss1, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
dog_imag_path = '/Users/edgarromo/anaconda3/envs/curso_ML19/practica_deep/cats_and_dogs_short/test/dogs/dog.1512.jpg'

In [ ]:
import numpy as np

In [ ]:
img = image.load_img(dog_imag_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

In [ ]:
plt.imshow(img_tensor[0])
plt.show()

In [ ]:
preds = model.predict(img_tensor)

In [ ]:
preds

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
#model = load_model('/content/drive/My Drive/cats_and_dogs_small_2.h5')  # para el caso de drive
model = load_model('/Users/edgarromo/anaconda3/envs/curso_ML19/practica_deep/1cats_and_dogs_small_2.h5')

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

In [ ]:
 model.evaluate_generator(test_generator, steps=100,
                   callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=1)